<a href="https://colab.research.google.com/github/aysavs/COVID-19/blob/main/Xception_CT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Veri Kümesine erişmek için Google Drive'a bağlanma




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/COVID-19'

Mounted at /content/drive
/content/drive/My Drive/COVID-19


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tüm kütüphane ve gereklilikleri içe aktarma

In [ ]:
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import cv2
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical




### Bazı Parametrelerin Tanımlanması

In [ ]:
#görüntülerin yeniden boyutlandırma
IMAGE_SIZE = [224, 224] # veri kümesine bağlı olarak değişiklik yapılabilir

# eğitim yapılandırması:
epochs = 500
batch_size = 32

#yolları tanımla
covid_path = 'data/ct/CT_COVID'
noncovid_path = 'data/ct/CT_NonCOVID'

# Path.jpg veya jpeg türü görüntü almak için glob'u kullanma
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [ ]:
# Dosya değişkenlerinin içeriğini görselleştirme
print("İlk 5 Covıd Dosyası: ",covid_files[0:5])
print("Toplam Sayı: ",len(covid_files))
print("İlk 5 NonCovıd Dosyası: ",noncovid_files[0:5])
print("Toplam Sayı: ",len(noncovid_files))

İlk 5 Covıd Dosyası:  ['data/ct/CT_COVID/2020.03.19.20038539-p10-61.png', 'data/ct/CT_COVID/2020.03.22.20040782-p25-1544.png', 'data/ct/CT_COVID/2020.02.23.20026856-p17-115_0.png', 'data/ct/CT_COVID/2020.03.01.20029769-p21-73_2_0.png', 'data/ct/CT_COVID/2020.02.24.20027052-p8-73_0.png']
Toplam Sayı:  349
İlk 5 NonCovıd Dosyası:  ['data/ct/CT_NonCOVID/781.png', 'data/ct/CT_NonCOVID/1310.png', 'data/ct/CT_NonCOVID/782.png', 'data/ct/CT_NonCOVID/1702.png', 'data/ct/CT_NonCOVID/921.png']
Toplam Sayı:  397


### Dosyalardan Resim ve Sınıf Etiketleri Getirme

```
```



In [ ]:
# Dosyalardan Resim ve Sınıf Etiketleri Getirme
covid_labels = []
noncovid_labels = []

covid_images=[]
noncovid_images=[]

for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i]) # dosya okuma
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # biçimi keras'a göre düzenleme
  image = cv2.resize(image,(224,224)) # modele göre yeniden boyutlandırma
  covid_images.append(image) # resim ekle
  covid_labels.append('CT_COVID') #sınıf etiketi ekle
for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  noncovid_images.append(image)
  noncovid_labels.append('CT_NonCOVID')

### Veri kümesinden İlk 40 Görüntüyü Görselleştirme

In [ ]:
# rastgele bir resime bakma
def plot_images(images, title):
    nrows, ncols = 5, 8
    figsize = [10, 6]

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, facecolor=(1, 1, 1))

    for i, axi in enumerate(ax.flat):
        axi.imshow(images[i])
        axi.set_axis_off()

    plt.suptitle(title, fontsize=24)
    plt.tight_layout(pad=0.2, rect=[0, 0, 1, 0.9])
    plt.show()
plot_images(covid_images, 'Pozitif COVID-19 CT Taramaları')
plot_images(noncovid_images, 'Negatif COVID-19 CT Taramaları')

### **Normalizasyon**
#### Model, görüntüleri piksel dizisi şeklinde alır. Bu nedenle pikselleri diziye dönüştürülür ve normalleştirilir

In [ ]:
#Diziye dönüştürme ve [0,1] aralığına normalleştirme
covid_images = np.array(covid_images) / 255
noncovid_images = np.array(noncovid_images) / 255

### **Eğitim ve Test İçin Veri Setinin Bölünmesi**

In [ ]:
# Her iki görüntü türü için eğitim ve test setlerine ayırma
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(
    covid_images, covid_labels, test_size=0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(
    noncovid_images, noncovid_labels, test_size=0.2)

# Her iki görüntü türü için birleştirme kümeleri
X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)

# Etiketleri kategorilere ayırma  model için 0 veya 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

### Eğitim ve Test setlerinden birkaç görüntüyü görselleştirme

In [ ]:
plot_images(covid_x_train, 'X_egitim')
plot_images(covid_x_test, 'X_test')

# # y_train and_test, X_train ve X_test için COVID ve noncovıd'yi temsil eden 0 ve 1 sınıf etiketlerini içerir

### **Modeli Oluşturma Ve Görselleştirme**

In [ ]:
# Modeli Oluşturma
xception = Xception(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

outputs = xception.output
outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation="softmax")(outputs)

model = Model(inputs=xception.input, outputs=outputs)

for layer in xception.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
)

In [ ]:
# Modeli Görselleştirme
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           


### Görüntü Büyütme
Görüntüleri eğitmek için farklı konumlarda, açılarda, vb. çevirmel

In [ ]:
train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


### Modeli Eğitmek

In [ ]:
history = model.fit(train_aug.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_test, y_test),
                    validation_steps=len(X_test) / batch_size,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=epochs)

In [ ]:
# Modeli ve Ağırlıkları Kaydetme
model.save('xception_ct.h5')
model.save_weights('xception_weights_ct.hdf5')

In [ ]:
# Kaydedilen modeli yükleme
model = load_model('xception_ct.h5')

### **Tahminlerde Bulunma**

In [ ]:
y_pred = model.predict(X_test, batch_size=batch_size)

### İlk 10 tahminin görselleştirilmesi

In [ ]:
prediction=y_pred[0:10]
for index, probability in enumerate(prediction):
  if probability[1] > 0.5:
        plt.title('%.2f' % (probability[1]*100) + '% COVID')
  else:
        plt.title('%.2f' % ((1-probability[1])*100) + '% NonCOVID')
  plt.imshow(X_test[index])
  plt.show()

In [ ]:
# Binary dönüştürme
y_pred_bin = np.argmax(y_pred, axis=1)
y_test_bin = np.argmax(y_test, axis=1)

###  ROC Eğrisi Çizimi

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_bin, y_pred_bin)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title(' ROC eğrisi ')
plt.xlabel('False Positive')
plt.ylabel('True Positive')
plt.grid(True)

NameError: ignored

###  Karışıklık Matrisi Grafiği

In [ ]:
def plot_confusion_matrix(normalize):
  classes = ['COVID','NonCOVID']
  tick_marks = [0.5,1.5]
  cn = confusion_matrix(y_test_bin, y_pred_bin,normalize=normalize)
  sns.heatmap(cn,cmap='plasma',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Karışıklık Matrisi')
  plt.ylabel('Dogruluk')
  plt.xlabel('Tahmin')
  plt.show()

print('Normalleştirme olmadan Karışıklık Matrisi')
plot_confusion_matrix(normalize=None)

print('Normalleştirilmiş Değerlerle Karışıklık Matrisi')
plot_confusion_matrix(normalize='true')

### Sınıflandırma Raporu

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))

              precision    recall  f1-score   support

           0       0.56      0.86      0.67        70
           1       0.76      0.40      0.52        80

    accuracy                           0.61       150
   macro avg       0.66      0.63      0.60       150
weighted avg       0.67      0.61      0.59       150




### **Doğruluk ve Kayıp Grafiği**

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Dogruluk Modeli')
plt.ylabel('Dogruluk')
plt.xlabel('Epoch')

plt.legend(['Egtim', 'Test'])
plt.savefig('xception_ct_accuracy.png')
plt.show()


In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Kayıp Modeli')
plt.ylabel('Kayıp')
plt.xlabel('Epoch')

plt.legend(['Eğitim', 'Test'])
plt.savefig('xception_ct_loss.png')
plt.show()